In [ ]:
from magnet.base import Magnet
from magnet.base import EmbeddedMagnet

cluster = EmbeddedMagnet()
cluster.start()
magnet = cluster.create_magnet()
await magnet.align()

config = {
    "host": "localhost",
    "credentials": None,
    "domain": None,
    "stream_name": "my_stream",
    "category": "my_category",
    "kv_name": "my_kv",
    "session": "my_session",
    "os_name": "my_object_store",
    "index": {
        "milvus_uri": "127.0.0.1",
        "milvus_port": 19530,
        "milvus_user": "test",
        "milvus_password": "test",
        "dimension": 1024,
        "model": "BAAI/bge-large-en-v1.5",
        "name": "test",
        "options": {
            'metric_type': 'COSINE',
            'index_type':'HNSW',
            'params': {
                "efConstruction": 40
                , "M": 48
            }
        }
    }
}

magnet = Magnet(config)
await magnet.align()

In [ ]:
from magnet.ic.field import Charge

field = Charge(magnet)
await field.on()

In [ ]:
# simple data pipelines with predictable outcomes
from magnet.utils.data_classes import FilePayload
import base64

file = "/Users/dylan/VSCode/FinPDF/taking-stock-quarterly-outlook-en-us.pdf_2024-07-04.pdf"

with open(file, "rb") as fh:
    content = fh.read()
    encoded_content = base64.b64encode(content).decode('utf-8')

await field.pulse(
    FilePayload(encoded_content, file.split('/')[-1])
    , v=True
)

from magnet.ic.field import Resonator

reso = Resonator(magnet)

async def handle_payload(payload, msg):
    print(msg)
    await reso.download(msg)
    await payload.delete(msg.name)

await reso.on(obj=True)

await reso.listen(cb=handle_payload, v=True)